In [18]:

from openpyxl import load_workbook
workbook = load_workbook(filename='../etc/M1M Modbus map V1.2.xlsx')

from string import Template
ai_template = Template(
    '''
    record(ai, "${pv_name}") {
        field(DESC, "")
        field(EGU, "${unit}")
        field(DTYP,"asynInt32")
        field(INP,"@asyn($(PORT), ${register_addr})${data_type}")
        field(ASLO, "${resolution}")
        field(SCAN,"2 second")
    }
    '''
)

sheet = workbook['M1M registers map']
sheet.cell(row=10, column=6).value

ColNames = {}
Current  = 0
for COL in sheet.iter_cols(1, sheet.max_column):
    ColNames[COL[0].value] = Current
    Current += 1

In [ ]:
# Write db file

def to_camel(string: str):
    if type(string) != None:
        output = ''
        for _ in range(len(string)):
            if _>0 and string[_-1]==' ':
                output += string[_].upper()
            else:
                output += string[_]
        output = output.replace(" ", "")
        return output

with open('../ioc/database/st.db', 'w') as file:

    for row_cells in sheet.iter_rows(2, sheet.max_row):
        
        if row_cells[ColNames['Monitored']].value == 'yes':
            
            print(row_cells[ColNames['Unit']].value)
            
            if row_cells[ColNames['Quantity/Functionality']].value != None:
                
                if row_cells[ColNames['Data Type']].value == 'Unsigned' and row_cells[ColNames['Length']].value == 2:
                
                    file.write(ai_template.safe_substitute(
                        pv_name = to_camel(row_cells[ColNames['Quantity/Functionality']].value),
                        register_addr = row_cells[ColNames['Register\n(Dec)']].value,
                        data_type = '',
                        resolution = row_cells[ColNames['Resolution']].value,
                        unit = row_cells[ColNames['Unit']].value
                        ))
                
                elif row_cells[ColNames['Data Type']].value == 'Unsigned' and row_cells[ColNames['Length']].value == 1:
                    
                    file.write(ai_template.safe_substitute(
                        pv_name = to_camel(row_cells[ColNames['Quantity/Functionality']].value),
                        register_addr = row_cells[ColNames['Register\n(Dec)']].value,
                        data_type = 'INT16',
                        resolution = row_cells[ColNames['Resolution']].value,
                        unit = row_cells[ColNames['Unit']].value if row_cells[ColNames['Unit']].value != None else ''
                        ))
                
                elif row_cells[ColNames['Data Type']].value == 'Signed' and row_cells[ColNames['Length']].value == 2:
                    
                    file.write(ai_template.safe_substitute(
                        pv_name = to_camel(row_cells[ColNames['Quantity/Functionality']].value),
                        register_addr = row_cells[ColNames['Register\n(Dec)']].value,
                        data_type = 'INT32_BE',
                        resolution = row_cells[ColNames['Resolution']].value,
                        unit = row_cells[ColNames['Unit']].value if row_cells[ColNames['Unit']].value != None else ''
                        ))
                
                elif row_cells[ColNames['Data Type']].value == 'Signed' and row_cells[ColNames['Length']].value == 1:
                    
                    file.write(ai_template.safe_substitute(
                        pv_name = to_camel(row_cells[ColNames['Quantity/Functionality']].value),
                        register_addr = row_cells[ColNames['Register\n(Dec)']].value,
                        data_type = 'INT32_BE',
                        resolution = row_cells[ColNames['Resolution']].value,
                        unit = row_cells[ColNames['Unit']].value if row_cells[ColNames['Unit']].value != None else ''
                        ))


In [ ]:
# Testing serial

import serial, time

ser = serial.Serial('/dev/pts/5', baudrate=9600, bytesize=serial.EIGHTBITS, parity=serial.PARITY_NONE)
ser.write(b'\x02\x03\x5B\x02\x00\x02\x76\xDC')

size = 0
answer = ''
# while size < 9:
#     print(ser.read())
#     size +=1
# print(answer)

time.sleep(1)
print(ser.readline())

ser.close()

In [ ]:
from pymodbus.client import ModbusSerialClient

client = ModbusSerialClient(
    method='rtu',
    port='/dev/pts/14',
    baudrate=115200,
    timeout=3,
    parity='N',
    stopbits=1,
    bytesize=8
)

if client.connect():  # Trying for connect to Modbus Server/Slave
    '''Reading from a holding register with the below content.'''
    res = client.read_holding_registers(address=23298, count=2, slave=2)

    '''Reading from a discrete register with the below content.'''
    # res = client.read_discrete_inputs(address=1, count=1, unit=1)

    if not res.isError():
        print(res.registers)
    else:
        print(res)

else:
    print('Cannot connect to the Modbus Server/Slave')